# Download Dataset

In [68]:
# Step 1: Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib

def mean_absolute_percentage_error(y_true, y_pred):
    return 100 * (abs((y_true - y_pred) / y_true).mean())



In [69]:
# โหลดข้อมูล
file_path = "CSV/cooling_system_data.csv"
data = pd.read_csv(file_path)
data.head()

,Flow Rate (L/min),Inlet Temperature (°C),Outlet Temperature (°C),Delta Temperature (°C),Pressure (Bar),Delta Pressure (Bar),Power Consumption (kW),Vibration (m/s²),Ambient Temperature (°C),Time of Day,Cooling Load (kW),Motor Speed (RPM),Regular RPM (RPM),Energy Usage (Regular RPM) (kW),Optimal RPM,Energy Usage (Optimal RPM) (kW)
0,1061.810178,37.872710,32.472816,9.996702,3.919993,0.136948,35.525783,1.257846,29.483681,Off-Peak,23.999361,1500.311851,1500,28.253678,1275.265073,19.777574
1,1926.071460,39.468713,31.658242,12.467468,1.738048,0.124301,28.371698,0.173917,26.422267,Off-Peak,67.230245,1619.137152,1500,21.260912,1376.266579,14.882639
2,1597.990913,47.090948,28.523078,10.626668,2.386559,0.341677,48.579941,1.263295,26.895388,Peak,73.628290,1373.243702,1500,38.659098,1167.257146,27.061369
3,1397.987726,36.800088,37.145333,5.833026,3.653123,0.486447,18.759138,0.270370,27.710067,Off-Peak,69.652271,1257.311660,1500,49.705205,1068.714911,34.793644
4,734.027961,47.392994,34.532483,6.855802,2.928357,0.301089,33.514257,1.449654,28.054800,Off-Peak,63.318148,1430.960334,1500,34.047603,1216.316284,23.833322


In [70]:
# Data Preprocessing
# Encode categorical variable 'Time of Day' to numerical
data['Time of Day'] = data['Time of Day'].map({'Peak': 1, 'Off-Peak': 0})

# Train Model - Optimal RPM

In [71]:
# Feature และ Target
features = [
    "Flow Rate (L/min)", "Inlet Temperature (°C)", "Outlet Temperature (°C)",
    "Delta Temperature (°C)", "Pressure (Bar)", "Delta Pressure (Bar)",
    "Power Consumption (kW)", "Vibration (m/s²)", "Ambient Temperature (°C)",
    "Time of Day", "Cooling Load (kW)", "Motor Speed (RPM)",
    "Energy Usage (Regular RPM) (kW)"
]
target = "Optimal RPM"

X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [72]:
# Step 5: Scale Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# บันทึก scaler ที่สร้างไว้ในไฟล์ .pkl
with open("scaler_optimalRPM.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [73]:

# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Support Vector Regressor": SVR(),
    "Neural Network (MLP)": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
}

# Dictionary to store results
results = {}


In [74]:
for model_name, model in models.items():
    # Train the model
    model.fit(X_train_scaled, y_train)
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=True) ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[model_name] = {
        "Mean Squared Error (MSE)": mse,
        "Root Mean Squared Error (RMSE)": rmse,
        "Mean Absolute Error (MAE)": mae,
        "Mean Absolute Percentage Error (MAPE)": mape,
        "R^2 Score": r2
    }


# Convert results to a DataFrame
results_df = pd.DataFrame(results).T

# Step 8: Display Results
print("Model Comparison Results:")
print(results_df)
results_df.to_csv("model_comparison_results.csv", index=True)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use 

Model Comparison Results:
                          Mean Squared Error (MSE)  \
Linear Regression                        11.929546   
Decision Tree                             0.137499   
Random Forest                             0.034685   
Support Vector Regressor              16070.998677   
Neural Network (MLP)                     10.325446   

                          Root Mean Squared Error (RMSE)  \
Linear Regression                               3.453917   
Decision Tree                                   0.370809   
Random Forest                                   0.186238   
Support Vector Regressor                      126.771443   
Neural Network (MLP)                            3.213323   

                          Mean Absolute Error (MAE)  \
Linear Regression                          1.596159   
Decision Tree                              0.263492   
Random Forest                              0.138202   
Support Vector Regressor                 107.425118   
Neural Networ

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [75]:
# Use Random Forest for energy calculation
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

# Add Regular RPM and Energy Usage (Regular RPM) to Test Set
X_test = X_test.copy()
X_test["Regular RPM"] = data.loc[X_test.index, "Regular RPM (RPM)"]
X_test["Energy Usage (Regular RPM)"] = data.loc[X_test.index, "Energy Usage (Regular RPM) (kW)"]

# Calculate Energy Usage (Predicted Optimal RPM)
X_test["Optimal RPM (Predicted)"] = y_pred_rf
X_test["Energy Usage (Predicted Optimal RPM)"] = (
    X_test["Energy Usage (Regular RPM)"] * (X_test["Optimal RPM (Predicted)"] / X_test["Regular RPM"])
)

# Calculate Energy Reduction (%)
X_test["Energy Reduction (%)"] = (
    (X_test["Energy Usage (Regular RPM)"] - X_test["Energy Usage (Predicted Optimal RPM)"]) /
    X_test["Energy Usage (Regular RPM)"]
) * 100

# Summarize results
avg_reduction_rf = X_test["Energy Reduction (%)"].mean()
avg_energy_predicted = X_test["Energy Usage (Predicted Optimal RPM)"].mean()
avg_energy_regular = X_test["Energy Usage (Regular RPM)"].mean()

# Print summary
print("\nRandom Forest Results:")
print(f"Average Energy Reduction (%): {avg_reduction_rf:.2f}%")
print(f"Predicted Energy Usage (Optimal RPM): {avg_energy_predicted:.2f} kW")
print(f"Original Energy Usage (Regular RPM): {avg_energy_regular:.2f} kW")

# Save Random Forest results to CSV
output_file = "random_forest_optimal_rpm_results.csv"
X_test[[
    "Optimal RPM (Predicted)", 
    "Energy Usage (Predicted Optimal RPM)", 
    "Energy Reduction (%)"
]].to_csv(output_file, index=False)

print(f"Random Forest results saved to {output_file}")


Random Forest Results:
Average Energy Reduction (%): 20.31%
Predicted Energy Usage (Optimal RPM): 31.64 kW
Original Energy Usage (Regular RPM): 39.71 kW
Random Forest results saved to random_forest_optimal_rpm_results.csv


In [77]:
with open("random_forest_optimalRPM.pkl", "wb") as file:
    pickle.dump(rf_model, file)


## ตอนเอาไปใช้งาน

In [10]:
# # Load the model from the file
# loaded_rf_model = joblib.load(model_filename)

# # ใช้โมเดลที่โหลดมาในการพยากรณ์
# predictions = loaded_rf_model.predict(X_test_scaled)


# Train Model - Efficiency Percentage (%)

In [42]:
# โหลดข้อมูล
file_path = "CSV/cooling_system_data_with_new_targets.csv"
data = pd.read_csv(file_path)
data.head()

,Flow Rate (L/min),Inlet Temperature (°C),Outlet Temperature (°C),Delta Temperature (°C),Pressure (Bar),Delta Pressure (Bar),Power Consumption (kW),Vibration (m/s²),Ambient Temperature (°C),Time of Day,Cooling Load (kW),Motor Speed (RPM),Regular RPM (RPM),Energy Usage (Regular RPM) (kW),Optimal RPM,Energy Usage (Optimal RPM) (kW),Anomaly Detection Index (ADI),Efficiency Percentage (%),Remaining Lifespan (years)
0,1061.810178,37.872710,32.472816,9.996702,3.919993,0.136948,35.525783,1.257846,29.483681,Off-Peak,23.999361,1500.311851,1500,28.253678,1275.265073,19.777574,0.004849,94.926527,9.492653
1,1926.071460,39.468713,31.658242,12.467468,1.738048,0.124301,28.371698,0.173917,26.422267,Off-Peak,67.230245,1619.137152,1500,21.260912,1376.266579,14.882639,0.000762,99.415458,9.941546
2,1597.990913,47.090948,28.523078,10.626668,2.386559,0.341677,48.579941,1.263295,26.895388,Peak,73.628290,1373.243702,1500,38.659098,1167.257146,27.061369,0.008885,90.493244,9.049324
3,1397.987726,36.800088,37.145333,5.833026,3.653123,0.486447,18.759138,0.270370,27.710067,Off-Peak,69.652271,1257.311660,1500,49.705205,1068.714911,34.793644,0.007011,92.551690,9.255169
4,734.027961,47.392994,34.532483,6.855802,2.928357,0.301089,33.514257,1.449654,28.054800,Off-Peak,63.318148,1430.960334,1500,34.047603,1216.316284,23.833322,0.013024,85.947735,8.594774


In [43]:
data.columns

Index(['Flow Rate (L/min)', 'Inlet Temperature (°C)',
       'Outlet Temperature (°C)', 'Delta Temperature (°C)', 'Pressure (Bar)',
       'Delta Pressure (Bar)', 'Power Consumption (kW)', 'Vibration (m/s²)',
       'Ambient Temperature (°C)', 'Time of Day', 'Cooling Load (kW)',
       'Motor Speed (RPM)', 'Regular RPM (RPM)',
       'Energy Usage (Regular RPM) (kW)', 'Optimal RPM',
       'Energy Usage (Optimal RPM) (kW)', 'Anomaly Detection Index (ADI)',
       'Efficiency Percentage (%)', 'Remaining Lifespan (years)'],
      dtype='object')

In [44]:
# Feature และ Target
features = [
    "Flow Rate (L/min)", "Inlet Temperature (°C)", "Outlet Temperature (°C)",
    "Delta Temperature (°C)", "Pressure (Bar)", "Delta Pressure (Bar)",
    "Power Consumption (kW)", "Vibration (m/s²)", "Ambient Temperature (°C)",
    "Cooling Load (kW)", "Motor Speed (RPM)", "Energy Usage (Regular RPM) (kW)"
]
target = "Efficiency Percentage (%)"

X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [45]:
# Step 5: Scale Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# บันทึก scaler ที่สร้างไว้ในไฟล์ .pkl
with open("scale_Efficiency_Percentage.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [46]:
# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Support Vector Regressor": SVR(),
    "Neural Network (MLP)": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
}

# Dictionary to store results
results = {}


In [47]:
for model_name, model in models.items():
    # Train the model
    model.fit(X_train_scaled, y_train)
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=True) ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[model_name] = {
        "Mean Squared Error (MSE)": mse,
        "Root Mean Squared Error (RMSE)": rmse,
        "Mean Absolute Error (MAE)": mae,
        "Mean Absolute Percentage Error (MAPE)": mape,
        "R^2 Score": r2
    }


# Convert results to a DataFrame
results_df = pd.DataFrame(results).T

# Step 8: Display Results
print("Model Comparison Results:")
print(results_df)
results_df.to_csv("model_comparison_results_ Efficiency_Percentage.csv", index=True)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use 

Model Comparison Results:
                          Mean Squared Error (MSE)  \
Linear Regression                        32.051080   
Decision Tree                             6.294640   
Random Forest                             1.578917   
Support Vector Regressor                 20.971307   
Neural Network (MLP)                      0.227329   

                          Root Mean Squared Error (RMSE)  \
Linear Regression                               5.661367   
Decision Tree                                   2.508912   
Random Forest                                   1.256550   
Support Vector Regressor                        4.579444   
Neural Network (MLP)                            0.476791   

                          Mean Absolute Error (MAE)  \
Linear Regression                          3.938266   
Decision Tree                              1.507824   
Random Forest                              0.730947   
Support Vector Regressor                   2.049489   
Neural Networ

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [48]:
import pickle

# Save the trained Neural Network model
nn_model = models["Neural Network (MLP)"]

# Save the model as a .pkl file
with open("nn_model_Efficiency_Percentage.pkl", "wb") as file:
    pickle.dump(nn_model, file)
print("Neural Network model saved as 'nn_model_Efficiency_Percentage.pkl'.")


Neural Network model saved as 'nn_model_Efficiency_Percentage.pkl'.


In [22]:
#code สำหรับ load มาใช้

# # Load the saved model
# with open("nn_model_Efficiency_Percentage.pkl", "rb") as file:
#     loaded_nn_model = pickle.load(file)
# print("Neural Network model loaded successfully.")


# Train Model - Remaining Lifespan

In [32]:
# Feature และ Target
features = [
    "Delta Temperature (°C)", "Pressure (Bar)", "Delta Pressure (Bar)",
    "Power Consumption (kW)", "Cooling Load (kW)",
    "Motor Speed (RPM)", "Vibration (m/s²)", "Ambient Temperature (°C)"
]
target = "Remaining Lifespan (years)"

X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [34]:
# Step 5: Scale Features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# บันทึก scaler ที่สร้างไว้ในไฟล์ .pkl
with open("scale_Remaining_Lifespan.pkl", "wb") as file:
    pickle.dump(scaler, file)

In [35]:
# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Support Vector Regressor": SVR(),
    "Neural Network (MLP)": MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=1000, random_state=42)
}

# Dictionary to store results
results = {}


In [36]:
for model_name, model in models.items():
    # Train the model
    model.fit(X_train_scaled, y_train)
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=True) ** 0.5
    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[model_name] = {
        "Mean Squared Error (MSE)": mse,
        "Root Mean Squared Error (RMSE)": rmse,
        "Mean Absolute Error (MAE)": mae,
        "Mean Absolute Percentage Error (MAPE)": mape,
        "R^2 Score": r2
    }


# Convert results to a DataFrame
results_df = pd.DataFrame(results).T

# Step 8: Display Results
print("Model Comparison Results:")
print(results_df)
results_df.to_csv("model_comparison_results_ Efficiency_Percentage.csv", index=True)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use 

Model Comparison Results:
                          Mean Squared Error (MSE)  \
Linear Regression                         0.319100   
Decision Tree                             0.059386   
Random Forest                             0.013337   
Support Vector Regressor                  0.029451   
Neural Network (MLP)                      0.008787   

                          Root Mean Squared Error (RMSE)  \
Linear Regression                               0.564890   
Decision Tree                                   0.243693   
Random Forest                                   0.115487   
Support Vector Regressor                        0.171612   
Neural Network (MLP)                            0.093738   

                          Mean Absolute Error (MAE)  \
Linear Regression                          0.392906   
Decision Tree                              0.143207   
Random Forest                              0.066266   
Support Vector Regressor                   0.088955   
Neural Networ

/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [37]:
import pickle

# Save the trained Neural Network model
nn_model = models["Neural Network (MLP)"]

# Save the model as a .pkl file
with open("nn_model_Remaining_Lifespan.pkl", "wb") as file:
    pickle.dump(nn_model, file)
print("Neural Network model saved as 'nn_model_Remaining_Lifespan.pkl'.")


Neural Network model saved as 'nn_model_Remaining_Lifespan.pkl'.


In [ ]:
#code สำหรับ load มาใช้

# # Load the saved model
# with open("nn_model_Efficiency_Percentage.pkl", "rb") as file:
#     loaded_nn_model = pickle.load(file)
# print("Neural Network model loaded successfully.")
